In [ ]:
#this query aims to identify the approximate date of death of individuals in the care home cohort with missing death data from the primary care and social are databases. this is calculated if the end date for the social care episode andd observation period for the 
create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_ghost_death_date` as (
  with ghost_cohort as (
  SELECT
   person_id, 
   episodestartdate,
   episodestopdate,
   dod,
   birth_date,
   date_diff(episodestopdate,birth_date,year) as age_at_stop_date 
  FROM `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`
   where dod is null),
add_observation as (
   select
   g.*,
   o.observation_period_end_date,
   date_diff(o.observation_period_end_date,g.episodestopdate,month) as difference_observation_episodestop
   from ghost_cohort g
   left join `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.observation_period` o
   on g.person_id = o.person_id)
   select
   *
   from add_observation
   where add_observation.difference_observation_episodestop between -2 and 2
)